# WEEK 1: BFS, DFS - 8 Puzzle

In [ ]:
from collections import deque

class Node:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action

def get_neighbors(state):
    def swap(state, i, j):
        state = list(state)
        state[i], state[j] = state[j], state[i]
        return tuple(state)

    neighbors = []
    blank_index = state.index(0)
    row, col = divmod(blank_index, 3)

    moves = []
    if row > 0: moves.append((blank_index - 3, 'move up'))
    if row < 2: moves.append((blank_index + 3, 'move down'))
    if col > 0: moves.append((blank_index - 1, 'move left'))
    if col < 2: moves.append((blank_index + 1, 'move right'))

    for move_index, action in moves:
        next_state = swap(state, blank_index, move_index)
        neighbors.append((next_state, action))

    return neighbors

def bfs(start_state, goal_state):
    start_node = Node(start_state)
    if start_state == goal_state:
        return start_node
    frontier = deque([start_node])
    explored = set()
    explored.add(start_state)

    while frontier:
        node = frontier.popleft()
        for next_state, action in get_neighbors(node.state):
            if next_state not in explored:
                child_node = Node(next_state, node, action=action)
                if next_state == goal_state:
                    return child_node
                explored.add(next_state)
                frontier.append(child_node)

    return None

def dfs(start_state, goal_state):
    start_node = Node(start_state)
    if start_state == goal_state:
        return start_node
    frontier = [start_node]
    explored = set()
    explored.add(start_state)

    while frontier:
        node = frontier.pop()
        for next_state, action in get_neighbors(node.state):
            if next_state not in explored:
                child_node = Node(next_state, node, action=action)
                if next_state == goal_state:
                    return child_node
                explored.add(next_state)
                frontier.append(child_node)
    return None
def print_solution(node):
    path = []
    actions = []

    while node:
        path.append(node)
        if node.parent:
            actions.append(node.action)
        node = node.parent

    path.reverse()
    actions.reverse()


    print("Initial State:")
    print_state(path[0].state)

    for i in range(1, len(path)):
        print(f"Action: {actions[i-1]}")
        print_state(path[i].state)

def print_state(state):
    for i in range(0, 9, 3):
        print(" ".join(map(str, state[i:i+3])))
    print()

def is_solvable(state):
    inversions = 0
    flat_state = [tile for tile in state if tile != 0]
    for i in range(len(flat_state)):
        for j in range(i + 1, len(flat_state)):
            if flat_state[i] > flat_state[j]:
                inversions += 1
    return inversions % 2 == 0

if __name__ == "__main__":
    initial_state = (1, 2, 3, 4, 5, 6, 7, 8, 0)
    goal_state = (1, 2, 3, 4, 5, 6, 0, 7, 8)

    if is_solvable(initial_state) and is_solvable(goal_state):
        print("Solving with BFS:")
        solution_node_bfs = bfs(initial_state, goal_state)

        if solution_node_bfs:
            print("BFS Solution found:")
            print_solution(solution_node_bfs)
        else:
            print("BFS: No solution found.")


        print("Solving with DFS: ")
        solution_node_dfs = dfs(initial_state, goal_state)

        if solution_node_dfs:
            print("DFS Solution found:")
            print_solution(solution_node_dfs)
        else:
            print("DFS: No solution found.")
    else:
        print("The puzzle is not solvable.")


Solving with BFS:
BFS Solution found:
Initial State:
1 2 3
4 5 6
7 8 0

Action: move left
1 2 3
4 5 6
7 0 8

Action: move left
1 2 3
4 5 6
0 7 8

Solving with DFS: 
DFS Solution found:
Initial State:
1 2 3
4 5 6
7 8 0

Action: move left
1 2 3
4 5 6
7 0 8

Action: move left
1 2 3
4 5 6
0 7 8



# WEEK 2: UCS, IDS - 8 Puzzle


In [ ]:
import heapq


GOAL_STATE = [1, 2, 3, 4, 5, 6, 7, 8, 0]


def get_neighbors(state):
    neighbors = []
    blank_pos = state.index(0)
    row, col = divmod(blank_pos, 3)

    moves = {
        'up': (-1, 0),
        'down': (1, 0),
        'left': (0, -1),
        'right': (0, 1)
    }

    for move, (row_offset, col_offset) in moves.items():
        new_row, new_col = row + row_offset, col + col_offset
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_blank_pos = new_row * 3 + new_col
            new_state = state[:]
            new_state[blank_pos], new_state[new_blank_pos] = new_state[new_blank_pos], new_state[blank_pos]
            neighbors.append(new_state)

    return neighbors

def uniform_cost_search(start_state):
    # Priority queue with tuples of (cost, state, path)
    pq = []
    heapq.heappush(pq, (0, start_state, []))
    visited = set()
    visited.add(tuple(start_state))

    while pq:
        cost, state, path = heapq.heappop(pq)

        if state == GOAL_STATE:
            return path + [state], cost

        for neighbor in get_neighbors(state):
            if tuple(neighbor) not in visited:
                visited.add(tuple(neighbor))
                heapq.heappush(pq, (cost + 1, neighbor, path + [state]))

    return None, None  # If no solution is found

def print_puzzle(state):
    for i in range(0, 9, 3):
        print(state[i:i+3])
    print()

start_state = [1, 2, 3, 4, 5, 6, 0, 7, 8]
solution_path, total_cost = uniform_cost_search(start_state)

if solution_path:
    print("Solution found with cost:", total_cost)
    print("\nSteps to reach the goal state:")
    for step, state in enumerate(solution_path):
        print(f"Step {step}:")
        print_puzzle(state)
else:
    print("No solution found")


Solution found with cost: 2

Steps to reach the goal state:
Step 0:
[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

Step 1:
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Step 2:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



In [ ]:
from collections import deque

GOAL_STATE = [1, 2, 3, 4, 5, 6, 7, 8, 0]

def get_neighbors(state):
    neighbors = []
    blank_pos = state.index(0)
    row, col = divmod(blank_pos, 3)

    moves = {
        'up': (-1, 0),
        'down': (1, 0),
        'left': (0, -1),
        'right': (0, 1)
    }

    for move, (row_offset, col_offset) in moves.items():
        new_row, new_col = row + row_offset, col + col_offset
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_blank_pos = new_row * 3 + new_col
            new_state = state[:]
            new_state[blank_pos], new_state[new_blank_pos] = new_state[new_blank_pos], new_state[blank_pos]
            neighbors.append(new_state)
    return neighbors

def depth_limited_search(state, depth, path, visited):
    if state == GOAL_STATE:
        return path
    if depth == 0:
        return None

    visited.add(tuple(state))
    for neighbor in get_neighbors(state):
        if tuple(neighbor) not in visited:
            result = depth_limited_search(neighbor, depth - 1, path + [neighbor], visited)
            if result:
                return result
    visited.remove(tuple(state))
    return None

def iterative_deepening_search(start_state):
    depth = 0
    while True:
        visited = set()
        result = depth_limited_search(start_state, depth, [start_state],visited)
        if result:
            return result
        depth += 1

def print_puzzle(state):
    for i in range(0, 9, 3):
        print(state[i:i+3])
    print()

start_state = [1, 2, 3, 4, 5, 6, 0, 7, 8]
solution_path = iterative_deepening_search(start_state)

if solution_path:
    print("Solution found:")
    for step, state in enumerate(solution_path):
        print(f"Step {step}:")
        print_puzzle(state)
else:
    print("No solution found")


Solution found:
Step 0:
[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

Step 1:
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Step 2:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



# Week 3: A* search algorithm - 8 Puzzle

In [ ]:
import random
import time
import heapq
import sys
import itertools

class Puzzle8:
    def __init__(self, initial_state=None, goal_state=None):
        self.goal_state = goal_state if goal_state else (1, 2, 3, 4, 5, 6, 7, 8, 0)
        self.initial_state = initial_state if initial_state else self.generate_solvable_state()
        self.blank = 0
        self.directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    def generate_solvable_state(self):
        while True:
            state = list(range(9))
            random.shuffle(state)
            if self.is_solvable(state):
                return tuple(state)

    def is_solvable(self, state):
        inversions = 0
        one_d_state = [num for num in state if num != 0]
        for i in range(len(one_d_state)):
            for j in range(i + 1, len(one_d_state)):
                if one_d_state[i] > one_d_state[j]:
                    inversions += 1
        return inversions % 2 == 0

    def heuristic_misplaced_tiles(self, state):
        return sum(1 for i, tile in enumerate(state) if tile != self.goal_state[i] and tile != 0)

    def heuristic_manhattan_distance(self, state):
        distance = 0
        for i, tile in enumerate(state):
            if tile != 0:
                goal_pos = self.goal_state.index(tile)
                distance += abs(goal_pos % 3 - i % 3) + abs(goal_pos // 3 - i // 3)
        return distance

    def get_neighbors(self, state):
        neighbors = []
        zero_pos = state.index(self.blank)
        zero_row, zero_col = divmod(zero_pos, 3)

        for direction in self.directions:
            new_row, new_col = zero_row + direction[0], zero_col + direction[1]
            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_zero_pos = new_row * 3 + new_col
                new_state = list(state)
                new_state[zero_pos], new_state[new_zero_pos] = new_state[new_zero_pos], new_state[zero_pos]
                neighbors.append(tuple(new_state))

        return neighbors

    def check_heuristic_consistency(self, heuristic):
        for state in itertools.permutations(range(9)):
            if self.is_solvable(state):
                current_heuristic = heuristic(state)
                for neighbor in self.get_neighbors(state):
                    neighbor_heuristic = heuristic(neighbor)
                    if current_heuristic > 1 + neighbor_heuristic:
                        return False
        return True

    def a_star_search(self, heuristic):
        start_time = time.time()
        open_set = [(heuristic(self.initial_state), 0, self.initial_state, [])]
        closed_set = set()
        steps = 0

        while open_set:
            _, cost, current_state, current_path = heapq.heappop(open_set)
            steps += 1

            if current_state == self.goal_state:
                cpu_time = time.time() - start_time
                return {
                    'steps': steps,
                    'cpu_time': cpu_time,
                    'path': current_path + [current_state],
                    'memory': sys.getsizeof(open_set) + sys.getsizeof(closed_set)
                }

            closed_set.add(current_state)

            for neighbor in self.get_neighbors(current_state):
                if neighbor not in closed_set:
                    heapq.heappush(open_set, (cost + 1 + heuristic(neighbor), cost + 1, neighbor, current_path + [current_state]))

        return None

def print_state_2d(state):
    for i in range(3):
        print(" ".join(str(state[i * 3 + j]) for j in range(3)))
    print()

if __name__ == "__main__":
    random.seed(42)
    puzzle = Puzzle8()

    print("Initial State:")
    print_state_2d(puzzle.initial_state)

    print("Goal State:")
    print_state_2d(puzzle.goal_state)

    heuristics = {
        "Manhattan Distance": puzzle.heuristic_manhattan_distance,
        "Misplaced Tiles": puzzle.heuristic_misplaced_tiles
    }

    results = {}

    for name, heuristic_function in heuristics.items():
        if puzzle.check_heuristic_consistency(heuristic_function):
            print(f"Heuristic '{name}' is consistent.")
            result = puzzle.a_star_search(heuristic_function)
            results[name] = result
        else:
            print(f"Heuristic '{name}' is not consistent.")

    print("\nResults Comparison:")
    print("{:<20} {:<15} {:<15} {:<15} {:<15}".format(
        "Heuristic", "Steps Taken", "CPU Time (s)", "Memory (bytes)", "Path Length"))
    print("-" * 80)

    for name, result in results.items():
        if result:
            print("{:<20} {:<15} {:<15.6f} {:<15} {:<15}".format(
                name, result['steps'], result['cpu_time'], result['memory'], len(result['path'])))
        else:
            print("{:<20} {:<15} {:<15} {:<15} {:<15}".format(name, "N/A", "N/A", "N/A", "N/A"))


Initial State:
2 5 7
8 6 1
4 0 3

Goal State:
1 2 3
4 5 6
7 8 0

Heuristic 'Manhattan Distance' is consistent.
Heuristic 'Misplaced Tiles' is consistent.

Results Comparison:
Heuristic            Steps Taken     CPU Time (s)    Memory (bytes)  Path Length    
--------------------------------------------------------------------------------
Manhattan Distance   409             0.003637        35184           22             
Misplaced Tiles      6362            0.044138        557552          22             


practice

In [ ]:
import heapq

# def manhattan_distance(state, goal):
#     distance = 0
#     for i in range(1, 9):  # We ignore the 0 tile (the blank space)
#         xi, yi = divmod(state.index(i), 3)
#         xg, yg = divmod(goal.index(i), 3)
#         distance += abs(xi - xg) + abs(yi - yg)
#     return distance

def misplaced_tiles(state, goal):
    return sum(1 for i in range(1, 9) if state[i] != goal[i])

def get_neighbors(state):
    neighbors = []
    index = state.index(0)
    row, col = divmod(index, 3)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    for dr, dc in directions:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_index = new_row * 3 + new_col
            new_state = state[:]
            new_state[index], new_state[new_index] = new_state[new_index], new_state[index]
            neighbors.append(new_state)
    return neighbors

def a_star(start, goal):
    # Initialize the priority queue with the start state
    pq = [(misplaced_tiles(start, goal), start, [])]  # (f, current_state, path)
    visited = set()

    while pq:
        f, current_state, path = heapq.heappop(pq)

        if tuple(current_state) in visited:
            continue

        visited.add(tuple(current_state))

        if current_state == goal:
            return path + [current_state]

        for neighbor in get_neighbors(current_state):
            if tuple(neighbor) not in visited:
                # Directly calculate f as (path length + 1) + heuristic
                new_f = len(path) + 1 + misplaced_tiles(neighbor, goal)
                heapq.heappush(pq, (new_f, neighbor, path + [current_state]))

    return None

start = [1, 2, 3, 4, 5, 6, 7, 0, 8]
goal = [1, 2, 3, 4, 5, 6, 7, 8, 0]


solution = a_star(start, goal)

if solution:
    print(f"Solution found using Heuristic:")
    for step in solution:
        print(step[:3])
        print(step[3:6])
        print(step[6:])
        print()
else:
    print("No solution found.")


Solution found using Heuristic:
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



# Week 4: Missionaries and Cannibals problem

### Justification for Using BFS


*   By using BFS, we ensure a complete, optimal, and efficient solution given the uniform action costs and the need to explore all valid possibilities to reach the goal state.
*  DFS is not suitable because it can get stuck in deep branches, leading to potentially infinite loops or non-optimal solutions. DFS does not guarantee finding the shortest path, which is essential in this problem.

*   UCS is ideal when costs differ between actions. However, since all actions here have the same cost (moving the boat with passengers), BFS suffices, and UCS offers no advantage over BFS in this case.



###State Representation:

A state in this problem can be represented as a tuple: (m_left, c_left, boat).
* m_left: Number of missionaries on the left bank.
* c_left: Number of cannibals on the left bank.
* boat: Position of the boat (0 for the left bank, 1 for the right bank).

### Cost of each action:
* Each action of shifting 2 people from the left bank to the right bank and vice-versa can be considered 1.

### Transition Function:
* The *get_successors* function generates possible valid states by transporting missionaries and/or cannibals to the opposite bank, considering the boat's current position.
* no of possible actions as 5 (1M0C, 2M0C, 1M1C, 0M1C, 0M2C)





In [ ]:
from collections import deque

def is_valid_state(m, c, boat):
    if m < 0 or c < 0 or m > 3 or c > 3:
        return False
    if (m > 0 and m < c) or (3 - m > 0 and 3 - m < 3 - c):
        return False
    return True

def get_successors(state):
    m_left, c_left, boat = state
    m_right, c_right = 3 - m_left, 3 - c_left
    successors = []

    actions = [(1, 0), (2, 0), (1, 1), (0, 1), (0, 2)]

    if boat == 0:
        for m, c in actions:
            new_m_left = m_left - m
            new_c_left = c_left - c
            if is_valid_state(new_m_left, new_c_left, 1):
                successors.append((new_m_left, new_c_left, 1))
    else:
        for m, c in actions:
            new_m_left = m_left + m
            new_c_left = c_left + c
            if is_valid_state(new_m_left, new_c_left, 0):
                successors.append((new_m_left, new_c_left, 0))

    return successors


def bfs():
    initial_state = (3, 3, 0)
    goal_state = (0, 0, 1)

    queue = deque([(initial_state, [])])
    visited = set()
    visited.add(initial_state)

    while queue:
        current_state, path = queue.popleft()
        if current_state == goal_state:
            return path + [current_state]

        for successor in get_successors(current_state):
            if successor not in visited:
                visited.add(successor)
                queue.append((successor, path + [current_state]))

    return None

solution = bfs()
if solution:
    print("Solution found:")
    for step in solution:
        print(step)
else:
    print("No solution found.")


Solution found:
(3, 3, 0)
(2, 2, 1)
(3, 2, 0)
(3, 0, 1)
(3, 1, 0)
(1, 1, 1)
(2, 2, 0)
(0, 2, 1)
(0, 3, 0)
(0, 1, 1)
(1, 1, 0)
(0, 0, 1)


# Week 5: local search techniques such as Simulated Annealing and Genetic algorithms to solve the 8-queens problem

In [ ]:
import random
import math
import time

def calculate_non_attacking_pairs(board):
    size = len(board)
    count = 0
    for i in range(size):
        for j in range(i + 1, size):
            if board[i] != board[j] and abs(board[i] - board[j]) != abs(i - j):
                count += 1
    return count

def random_board(size=8):
    return list(range(size))

def random_move(board):
    new_board = board[:]
    i, j = random.sample(range(len(board)), 2)
    new_board[i], new_board[j] = new_board[j], new_board[i]
    return new_board

def simulated_annealing(initial_board, schedule, time_limit=10):
    start_time = time.time()
    current = initial_board
    current_value = calculate_non_attacking_pairs(current)

    t = 1
    while True:
        elapsed_time = time.time() - start_time
        if elapsed_time >= time_limit:
            return current

        T = schedule(t)
        if T == 0:
            return current

        next_board = random_move(current)
        next_value = calculate_non_attacking_pairs(next_board)

        deltaE = next_value - current_value

        if deltaE > 0 or random.random() < math.exp(deltaE / T):
            current = next_board
            current_value = next_value

        t += 1

def linear_schedule(t, T0=1, beta=0.0001):
    return max(T0 - beta * t, 0)

def exponential_schedule(t, T0=100, alpha=0.01):
    return T0 * math.exp(-alpha * t)

def logarithmic_schedule(t, T0=100, beta=0.1):
    return T0 / (1 + beta * t)

time_limits = [10, 50, 100]

initial_board = random_board()

for time_limit in time_limits:
    print(f"\nRunning simulated annealing with time limit of {time_limit} seconds...")
    solution_linear = simulated_annealing(initial_board, linear_schedule, time_limit)
    solution_exp = simulated_annealing(initial_board, exponential_schedule, time_limit)
    solution_log = simulated_annealing(initial_board, logarithmic_schedule, time_limit)

    print("Initial Board:", initial_board)
    print("Solution (Linear Schedule):", solution_linear)
    print("Non-attacking pairs (Linear):", calculate_non_attacking_pairs(solution_linear))
    print("Solution (Exponential Schedule):", solution_exp)
    print("Non-attacking pairs (Exponential):", calculate_non_attacking_pairs(solution_exp))
    print("Solution (Logarithmic Schedule):", solution_log)
    print("Non-attacking pairs (Logarithmic):", calculate_non_attacking_pairs(solution_log))



Running simulated annealing with time limit of 10 seconds...
Initial Board: [0, 1, 2, 3, 4, 5, 6, 7]
Solution (Linear Schedule): [4, 6, 1, 3, 7, 0, 2, 5]
Non-attacking pairs (Linear): 28
Solution (Exponential Schedule): [4, 6, 0, 3, 1, 7, 5, 2]
Non-attacking pairs (Exponential): 28
Solution (Logarithmic Schedule): [5, 2, 6, 1, 3, 7, 0, 4]
Non-attacking pairs (Logarithmic): 28

Running simulated annealing with time limit of 50 seconds...
Initial Board: [0, 1, 2, 3, 4, 5, 6, 7]
Solution (Linear Schedule): [2, 6, 1, 7, 4, 0, 3, 5]
Non-attacking pairs (Linear): 28
Solution (Exponential Schedule): [5, 3, 0, 4, 7, 1, 6, 2]
Non-attacking pairs (Exponential): 28
Solution (Logarithmic Schedule): [6, 3, 1, 4, 7, 0, 2, 5]
Non-attacking pairs (Logarithmic): 28

Running simulated annealing with time limit of 100 seconds...
Initial Board: [0, 1, 2, 3, 4, 5, 6, 7]
Solution (Linear Schedule): [7, 1, 3, 0, 6, 4, 2, 5]
Non-attacking pairs (Linear): 28
Solution (Exponential Schedule): [2, 4, 1, 7, 0, 6,

In [ ]:
import random
import math
random.seed(42)
def calculate_non_attacking_pairs(board):
    size = len(board)
    count = 0
    for i in range(size):
        for j in range(i + 1, size):
            if board[i] != board[j] and abs(board[i] - board[j]) != abs(i - j):
                count += 1
    return count

def random_move(board):
    new_board = board[:]
    i, j = random.sample(range(len(board)), 2)
    new_board[i], new_board[j] = new_board[j], new_board[i]
    return new_board

def simulated_annealing(initial_board, schedule, max_iterations=10000):
    current = initial_board
    current_value = calculate_non_attacking_pairs(current)

    t = 1
    while t <= max_iterations:
        T = schedule(t)
        if T == 0:
            return current

        next_board = random_move(current)
        next_value = calculate_non_attacking_pairs(next_board)

        deltaE = next_value - current_value

        if deltaE > 0 or random.random() < math.exp(deltaE / T):
            current = next_board
            current_value = next_value

        t += 1

    return current  # Return the best found solution after max iterations

def linear_schedule(t, T0=1, beta=0.0001):
    return max(T0 - beta * t, 0)
# index of the list : a row on the chessboard. The value at each index represents the column position of the queen in that row.
initial_board = [0, 1, 2, 3, 4, 5, 6, 7]


solution_linear = simulated_annealing(initial_board, linear_schedule)

print("Initial Board:", initial_board)
print("Solution (Linear Schedule):", solution_linear)
print("Non-attacking pairs (Linear):", calculate_non_attacking_pairs(solution_linear))

Initial Board: [0, 1, 2, 3, 4, 5, 6, 7]
Solution (Linear Schedule): [1, 4, 6, 3, 0, 7, 5, 2]
Non-attacking pairs (Linear): 28


In [ ]:
import random
import time

n = 8
population_size = 10

def generate_population():
    population = set()
    for _ in range(population_size):
        individual = tuple(random.sample(range(n), n))
        if fitness(individual) == 0:
            return individual
        population.add(individual)
    return population

def fitness(individual):
    attacking_pairs = 0
    for i in range(n):
        for j in range(i + 1, n):
            if individual[i] == individual[j] or abs(individual[i] - individual[j]) == j - i:
                attacking_pairs += 1
    return attacking_pairs

def random_selection(population):
    fitness_values = [fitness(individual) for individual in population]
    total_fitness = sum(fitness_values)
    probabilities = [f / total_fitness for f in fitness_values]
    return random.choices(list(population), weights=probabilities, k=1)[0]

# partially-mapped crossover (PMX) strategy
def crossover(x, y):
    n = len(x)
    crossover_point = random.randint(1, n - 1)
    child = list(x[:crossover_point])

    for element in y:
        if element not in child:
            child.append(element)
        if len(child) == n:
            break

    return tuple(child)

def mutate(individual):
    i, j = random.sample(range(len(individual)), 2)
    individual = list(individual)
    individual[i], individual[j] = individual[j], individual[i]
    return tuple(individual)

def genetic_algorithm(mutate_rate, max_time):
    start_time = time.time()
    population = generate_population()
    # if an optimal solution is found during the initial population generation, instead of returning a set, it returns that single solution as a tuple.
    if isinstance(population, tuple):
        return population

    while time.time() - start_time < max_time:
        new_population = set()
        for _ in range(population_size):
            parent1 = random_selection(population)
            parent2 = random_selection(population)
            child = crossover(parent1, parent2)

            if random.random() < mutate_rate:
                child = mutate(child)

            if fitness(child) == 0:
                return child

            new_population.add(child)

        if not new_population:
            break

        population = new_population

    return sorted(population, key=fitness)[0] if population else None

def print_result(result):
    for row in result:
        print('. ' * row + 'Q ' + '. ' * (len(result) - row - 1))

random.seed(42)
max_time = 10

print(f"\nExecution Time: {max_time} Sec")
result = genetic_algorithm(mutate_rate=0.01, max_time=max_time)

if result is None:
    print("No solution found")
else:
    print("Solution found:")
    print(f"Queen Positions: {result}"))
    print(f"\nFitness value of result: {fitness(result)}")
print("")



Execution Time: 10 Sec
Solution found:
Queen Positions: (7, 1, 4, 2, 0, 6, 3, 5)
. . . . . . . Q 
. Q . . . . . . 
. . . . Q . . . 
. . Q . . . . . 
Q . . . . . . . 
. . . . . . Q . 
. . . Q . . . . 
. . . . . Q . . 

Fitness value of result: 0



# Week 6 : Adversarial Search - Tic Tac Toe

In [ ]:
import math

def print_board(board):
    for row in board:
        print("|".join(row))
        print("-" * 5)

def check_winner(board):
    for row in board:
        if row[0] == row[1] == row[2] and row[0] != " ":
            return row[0]

    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] and board[0][col] != " ":
            return board[0][col]

    if board[0][0] == board[1][1] == board[2][2] and board[0][0] != " ":
        return board[0][0]

    if board[0][2] == board[1][1] == board[2][0] and board[0][2] != " ":
        return board[0][2]

    return None

def is_full(board):
    for row in board:
        if " " in row:
            return False
    return True

def evaluate(board):
    winner = check_winner(board)
    if winner == "X":
        return 1
    elif winner == "O":
        return -1
    else:
        return 0

def minimax(board, depth, is_maximizing):
    score = evaluate(board)

    if score == 1 or score == -1:
        return score
    if is_full(board):
        return 0

    if is_maximizing:
        best_score = -math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == " ":
                    board[i][j] = "X"
                    score = minimax(board, depth + 1, False)
                    board[i][j] = " "
                    best_score = max(score, best_score)
        return best_score
    else:
        best_score = math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == " ":
                    board[i][j] = "O"
                    score = minimax(board, depth + 1, True)
                    board[i][j] = " "
                    best_score = min(score, best_score)
        return best_score

def find_best_move(board):
    best_score = -math.inf
    move = (-1, -1)

    for i in range(3):
        for j in range(3):
            if board[i][j] == " ":
                board[i][j] = "X"
                score = minimax(board, 0, False)
                board[i][j] = " "
                if score > best_score:
                    best_score = score
                    move = (i, j)

    return move

def play_game():
    board = [[" " for _ in range(3)] for _ in range(3)]
    current_player = "Computer"

    for _ in range(9):
        print_board(board)
        if check_winner(board):
            break

        if current_player == "Computer":
            i, j = find_best_move(board)
            board[i][j] = "X"
            current_player = "Human"
        else:
            print("Your turn! Enter row and column (0-2):")
            i, j = map(int, input().split())
            if board[i][j] == " ":
                board[i][j] = "O"
                current_player = "Computer"
            else:
                print("Invalid move! Try again.")

        winner = check_winner(board)
        if winner:
            print_board(board)
            print(f"{winner} wins!")
            return

    print_board(board)
    if is_full(board):
        print("It's a draw!")
        return

play_game()



 | | 
-----
 | | 
-----
 | | 
-----
X| | 
-----
 | | 
-----
 | | 
-----
Your turn! Enter row and column (0-2):
0 2
X| |O
-----
 | | 
-----
 | | 
-----
X| |O
-----
X| | 
-----
 | | 
-----
Your turn! Enter row and column (0-2):
2 0
X| |O
-----
X| | 
-----
O| | 
-----
X| |O
-----
X|X| 
-----
O| | 
-----
Your turn! Enter row and column (0-2):
1 2
X| |O
-----
X|X|O
-----
O| | 
-----
X| |O
-----
X|X|O
-----
O| |X
-----
X wins!


In [ ]:
import math

def print_board(board):
    for row in board:
        print("|".join(row))
        print("-" * 5)

def check_winner(board):
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] and board[i][0] != " ":
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] and board[0][i] != " ":
            return board[0][i]

    if board[0][0] == board[1][1] == board[2][2] and board[0][0] != " ":
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] and board[0][2] != " ":
        return board[0][2]

    return None

def is_full(board):
    return all(cell != " " for row in board for cell in row)

def evaluate(board):
    winner = check_winner(board)
    if winner == "X":
        return 1
    elif winner == "O":
        return -1
    return 0

def minimax_alpha_beta(board, depth, is_maximizing, alpha, beta):
    score = evaluate(board)
    if score == 1 or score == -1 or is_full(board):
        return score

    if is_maximizing:
        best_score = -math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == " ":
                    board[i][j] = "X"
                    score = minimax_alpha_beta(board, depth + 1, False, alpha, beta)
                    board[i][j] = " "
                    best_score = max(score, best_score)
                    alpha = max(alpha, best_score)
                    if beta <= alpha:
                        break
        return best_score
    else:
        best_score = math.inf
        for i in range(3):
            for j in range(3):
                if board[i][j] == " ":
                    board[i][j] = "O"
                    score = minimax_alpha_beta(board, depth + 1, True, alpha, beta)
                    board[i][j] = " "
                    best_score = min(score, best_score)
                    beta = min(beta, best_score)
                    if beta <= alpha:
                        break
        return best_score

def find_best_move(board):
    best_score = -math.inf
    best_move = (-1, -1)

    for i in range(3):
        for j in range(3):
            if board[i][j] == " ":
                board[i][j] = "X"
                score = minimax_alpha_beta(board, 0, False, -math.inf, math.inf)
                board[i][j] = " "
                if score > best_score:
                    best_score = score
                    best_move = (i, j)

    return best_move

def play_game():
    board = [[" "]*3 for _ in range(3)]
    current_player = "Computer"

    for turn in range(9):
        print_board(board)

        winner = check_winner(board)
        if winner:
            print(f"{winner} wins!")
            return

        if is_full(board):
            print("It's a draw!")
            return

        if current_player == "Computer":
            i, j = find_best_move(board)
            board[i][j] = "X"
            current_player = "Human"
        else:
            print("Your turn! Enter row and column (0-2):")
            i, j = map(int, input().split())
            if board[i][j] == " ":
                board[i][j] = "O"
                current_player = "Computer"
            else:
                print("Invalid move, try again.")
                continue

    print_board(board)
    if not check_winner(board):
        print("It's a draw!")

play_game()


 | | 
-----
 | | 
-----
 | | 
-----
X| | 
-----
 | | 
-----
 | | 
-----
Your turn! Enter row and column (0-2):
1 0
X| | 
-----
O| | 
-----
 | | 
-----
X|X| 
-----
O| | 
-----
 | | 
-----
Your turn! Enter row and column (0-2):
1 1
X|X| 
-----
O|O| 
-----
 | | 
-----
X|X|X
-----
O|O| 
-----
 | | 
-----
X wins!


Alpha-Beta Pruning significantly improves the Minimax algorithm's efficiency by pruning unnecessary branches, cutting the time complexity from
𝑂(𝑏^𝑑)to 𝑂(𝑏^𝑑/2). This reduces the number of nodes evaluated, leading to up to a 50% speed improvement in games like Tic-Tac-Toe, though the space complexity remains
𝑂(𝑑)due to recursion depth. To maximize pruning, node reordering is essential, with strategically important moves—such as the center and corners—evaluated first. Prioritizing these moves increases the chances of early cut-offs, further speeding up the search. Using heuristics for dynamic move ordering can enhance pruning, resulting in more efficient decision-making and less overall resource usage.


# Week 7: CSP

In [ ]:
def selectUnassignedVar(graph, assignment, domain):
    retVar = None
    mrv = float('inf')
    for var in graph:
        if var not in assignment:
            if len(domain[var]) < mrv:
                mrv = len(domain[var])
                retVar = var
            elif len(domain[var]) == mrv:
                if len(graph[var]) > len(graph[retVar]):
                    retVar = var
    return retVar


def constraintCount(graph, var, val, assignment, domain):
    count = 0
    for neighbour in graph[var]:
        if neighbour not in assignment and val in domain[neighbour]:
            count += 1
    return count


def consistent(assignment, constraints):
    for var1, var2 in constraints:
        if var1 in assignment and var2 in assignment and assignment[var1] == assignment[var2]:
            return False
    return True


def revise(domain, x, y):
    revised = False
     # Iterate over a copy of domain[x] to avoid modifying the list while iterating
    for xVal in domain[x][:]:
      # Check if there exists any yVal in domain[y] such that yVal != xVal
        if not any(yVal != xVal for yVal in domain[y]):
          # If no such yVal exists, remove xVal from domain[x]
            domain[x].remove(xVal)
            revised = True
    return revised


def ac3(graph, domain, arcs=None):
    queue = [(x, y) for x in graph for y in graph[x]] if arcs is None else arcs
    while queue:
        x, y = queue.pop(0)
        if revise(domain, x, y):
            if len(domain[x]) == 0:
                return False
            for k in graph[x]:
                if k != y:
                    queue.append((k, x))
    return True


def inferences(graph, var, val, assignment, domain):
    arcs = [(x, var) for x in graph[var] if x not in assignment]
    return ac3(graph, domain, arcs)

def backtrack(graph, assignment, domain, constraints):
    if len(assignment) == len(graph): return assignment
    var = selectUnassignedVar(graph, assignment, domain)

    for val in sorted(domain[var], key=lambda v: constraintCount(graph, var, v, assignment, domain)):
        assignment[var] = val
        if consistent(assignment, constraints):
            if inferences(graph, var, val, assignment, domain):
                result = backtrack(graph, assignment, domain, constraints)
                if result is not None:
                    return result
        del assignment[var]
    return None


def solveCSP(graph, n_colors):
    domain = {var: ["C" + str(i) for i in range(1, n_colors + 1)] for var in graph}
    constraints = set((node, neighbour) for node in graph for neighbour in graph[node])
    return backtrack(graph, {}, domain, constraints)



graph = {
    "V1": {"V2", "V3", "V5"},
    "V2": {"V1", "V5", "V6"},
    "V3": {"V1", "V5", "V6"},
    "V4": {"V6"},
    "V5": {"V1", "V2", "V3", "V6"},
    "V6": {"V2", "V3", "V4", "V5"}
}
colors = 3
result = solveCSP(graph, colors)

print("Node -> Colour")
if result:
    for var in result:
        print(f"{var} -> {result[var]}")
else:
    print("No solution found")


Node -> Colour
V5 -> C1
V6 -> C2
V1 -> C2
V2 -> C3
V3 -> C3
V4 -> C1


In [ ]:
! wget -q https://mat.tepper.cmu.edu/COLOR/instances/instances.tar
! tar -xf instances.tar
! mkdir instances
! cp anna.col school1.col -d instances
! rm -rf *.col*
! rm -rf instances.tar

print("Graph instances downloaded and extracted successfully.")

Graph instances downloaded and extracted successfully.


In [ ]:
!pip install tabulate
import tabulate
from IPython.display import HTML, display
css = """
<style>
table {
    display: inline-block;
    text-align: left;
    border: solid 1px;
    margin: 5px;
}
th {
    border: solid 1px;
    text-align: right;
    padding: 5px 10px;
}
td {
    border: solid 1px;
    text-align: right;
    padding: 5px;
}
</style>
"""

In [ ]:
# @title ANNA.COL graph

with open("instances/anna.col", "r") as f:
    lines = f.readlines()
    lines = lines[4:]

graph = {}
for line in lines:
    line = line.strip().split()
    line = line[1:]
    if line[0] not in graph:
        graph[line[0]] = set()
    if line[1] not in graph:
        graph[line[1]] = set()
    graph[line[0]].add(line[1])
    graph[line[1]].add(line[0])

print("Graph: ", graph)

colors = 12
csp = CSP(graph, colors)
result = csp.solve()

result = list(sorted(result.items(), key=lambda x: int(x[0])))

table_data = []
for i in range(0, len(result), 20):
    data = tabulate.tabulate(result[i:i+20], headers=["Node", "Colour"], tablefmt="html")
    table_data.append(data)

display(HTML(css + "".join(table_data)))

Graph:  {'1': {'36'}, '36': {'52', '125', '115', '28', '135', '93', '20', '7', '1', '13', '137', '6', '16', '133', '76', '80', '106', '136', '116', '18', '108', '74', '138', '83', '103', '59', '66', '127', '37', '91', '89', '64', '81', '118', '45', '51', '21', '33', '11', '99', '71', '72', '77', '5', '26', '95', '85', '48', '100', '79'}, '2': {'45'}, '45': {'127', '82', '115', '36', '64', '6', '2', '95', '61', '138', '136', '74', '18'}, '3': {'74'}, '74': {'135', '75', '20', '7', '105', '6', '16', '117', '30', '44', '78', '116', '136', '18', '108', '138', '3', '36', '83', '14', '59', '119', '60', '92', '127', '91', '123', '89', '81', '45', '130', '68', '21', '33', '17', '11', '72', '99', '95', '85', '65', '29', '46', '19'}, '4': {'18'}, '18': {'28', '135', '41', '63', '87', '75', '7', '39', '13', '76', '6', '16', '31', '78', '44', '121', '80', '105', '53', '116', '74', '108', '136', '138', '113', '83', '36', '9', '103', '59', '132', '107', '88', '91', '123', '92', '127', '89', '8', '64

Node,Colour
1,C1
2,C1
3,C1
4,C2
5,C1
6,C6
7,C3
8,C3
9,C2
10,C2


In [ ]:
# @title SCHOOL1.COL graph

with open("instances/school1.col", "r") as f:
    lines = f.readlines()
    lines = lines[7:]

graph = {}
for line in lines:
    line = line.strip().split()
    line = line[1:]
    if line[0] not in graph:
        graph[line[0]] = set()
    if line[1] not in graph:
        graph[line[1]] = set()
    graph[line[0]].add(line[1])
    graph[line[1]].add(line[0])

print("Graph: ", graph)

colors = 42
csp = CSP(graph, colors)
result = csp.solve()

result = list(sorted(result.items(), key=lambda x: int(x[0])))

table_data = []
for i in range(0, len(result), 40):
    data = tabulate.tabulate(result[i:i+40], headers=["Node", "Colour"], tablefmt="html")
    table_data.append(data)

display(HTML(css + "".join(table_data)))

Graph:  {'1': {'333', '364', '269', '190', '259', '93', '86', '147', '192', '31', '139', '39', '201', '76', '286', '348', '332', '324', '74', '292', '331', '38', '97', '337', '2', '381', '50', '60', '278', '179', '66', '92', '244', '89', '326', '281', '145', '379', '382', '165', '154', '21', '149', '198', '245', '250', '276', '33', '17', '11', '255', '349', '365', '380', '54', '377', '293', '318', '372', '373', '77', '232', '258', '5', '85', '320', '345', '126', '231', '383', '191'}, '11': {'364', '177', '80', '374', '14', '225', '222', '307', '179', '43', '10', '340', '172', '90', '17', '380', '173', '318', '155', '98', '93', '147', '13', '371', '370', '170', '321', '331', '376', '145', '239', '4', '255', '304', '377', '306', '5', '258', '12', '95', '15', '19', '146', '152', '310', '259', '101', '7', '1', '368', '3', '174', '36', '92', '91', '45', '214', '250', '50', '365', '176', '373', '47', '35', '164', '385', '39', '6', '16', '175', '240', '324', '18', '261', '97', '9', '103', '21

Node,Colour
1,C18
2,C6
3,C6
4,C30
5,C4
6,C3
7,C8
8,C16
9,C7
10,C5


# Week 9: DPLL

In [ ]:
def dpll(cnf, assignment={}):
    if not cnf:
        return True, assignment
    if any([len(clause) == 0 for clause in cnf]):
        return False, {}
    for clause in cnf:
        if len(clause) == 1:
            literal = clause[0]
            return dpll(unit_propagate(cnf, literal), {**assignment, abs(literal): literal > 0})
    for literal in pure_literals(cnf):
        return dpll(unit_propagate(cnf, literal), {**assignment, abs(literal): literal > 0})
    literal = choose_literal(cnf)
    satisfiable, result = dpll(unit_propagate(cnf, literal), {**assignment, abs(literal): literal > 0})
    if satisfiable:
        return True, result
    return dpll(unit_propagate(cnf, -literal), {**assignment, abs(literal): literal < 0})

def unit_propagate(cnf, literal):
    simplified_cnf = []
    for clause in cnf:
        if literal in clause:
            continue
        new_clause = [l for l in clause if l != -literal]
        simplified_cnf.append(new_clause)
    return simplified_cnf

def pure_literals(cnf):
    literals = {literal for clause in cnf for literal in clause}
    pure = set()
    for literal in literals:
        if -literal not in literals:
            pure.add(literal)
    return pure

def choose_literal(cnf):
    for clause in cnf:
        for literal in clause:
            return literal
    return None

In [ ]:
cnf1 = [[1, -3, 4], [-1, 2, 5], [-2, -4, 6], [3, 5, -6], [-3, -5], [4, -1], [5, 2, -6], [6, -2], [1, -5, 4], [-1, -6, 3]]
cnf2 = [[1, 2, -3], [3, -4], [4, 5, 6], [-1, 4], [2, -5, 6], [-2, 5], [1, 6, -4], [-6, 3], [-3, -5, 1], [4, -2, 3]]
cnf3 = [[-1, 2, 3], [-2, -3], [1, 4, -5], [-4, 2, 5], [3, -6], [1, -6, -2], [6, 5, -1], [1, -3, 6], [4, -2, 1], [-6, 2, -5]]
cnf4 = [[1, 2], [-1, -2], [1, -3], [-1, 3], [3, 4], [-3, -4], [5], [-5, 6], [7], [-7]]

cnfs = [cnf1, cnf2, cnf3, cnf4]

for i, cnf in enumerate(cnfs, 1):
    satisfiable, assignment = dpll(cnf)
    print(f"CNF {i}: {'Satisfiable' if satisfiable else 'Unsatisfiable'}, Assignment: {assignment}")

CNF 1: Satisfiable, Assignment: {1: True, 4: True, 2: True, 6: True, 3: True, 5: False}
CNF 2: Satisfiable, Assignment: {1: True, 4: True, 3: True, 6: True, 5: True}
CNF 3: Satisfiable, Assignment: {1: False, 2: False, 4: True, 5: True, 6: False, 3: False}
CNF 4: Unsatisfiable, Assignment: {}


# Week 10 - Forward chaining for FOL

In [ ]:

rules = [
    {
        "conditions": ["Sells(x, y, z)","American(x)", "Weapon(y)", "Hostile(z, America)"],
        "result": "Criminal(x)"
    },
    {
        "conditions": ["Missile(y)"],
        "result": "Weapon(y)"
    }
]

known_facts = {
    "American(West)",
    "Missile(M1)",
    "Owns(Nono, M1)",
    "Sells(West, M1, Nono)",
    "Hostile(Nono, America)"
}

def forward_chaining(query):
    facts_to_process = list(known_facts)
    processed_facts = set()

    while facts_to_process:
        current_fact = facts_to_process.pop(0)
        if current_fact == query:
            return True
        if current_fact not in processed_facts:
            processed_facts.add(current_fact)
            for rule in rules:
                substitutions = check_conditions(rule["conditions"], processed_facts)

                if substitutions is not None:
                    new_fact = apply_substitution(rule["result"], substitutions)
                    if new_fact == query:
                        return True
                    if new_fact not in processed_facts:
                        facts_to_process.append(new_fact)

    return False

def check_conditions(conditions, processed_facts):
    substitutions = {}
    for condition in conditions:
        condition_matched = False
        for fact in processed_facts:
            subs = unify(condition, fact)
            if subs is not None:
                substitutions.update(subs)
                condition_matched = True
                break
        if not condition_matched:
            return None
    return substitutions

def unify(expression1, expression2):
    predicate1, terms1 = expression1.strip(")").split("(")
    predicate2, terms2 = expression2.strip(")").split("(")
    if predicate1 != predicate2:
        return None
    vars1 = terms1.split(", ")
    vars2 = terms2.split(", ")
    if len(vars1) != len(vars2):
        return None
    substitutions = {}
    for term1, term2 in zip(vars1, vars2):
        if term1 != term2:
            if term1.islower():
                substitutions[term1] = term2
            elif term2.islower():
                substitutions[term2] = term1
            else:
                return None  # Conflict between constants
    return substitutions

def apply_substitution(conclusion, substitutions):
    for var, sub in substitutions.items():
        conclusion = conclusion.replace(var, sub)
    return conclusion


# Query to check
query = "Criminal(West)"

# Run the forward-chaining algorithm
result = forward_chaining(query)
print(f"Is West a criminal? {result}")


Is West a criminal? True


# Week 11 - Probabilistic Graphical Model

In [ ]:
# Conditional Probability Tables (CPTs)
P_B = {True: 0.001, False: 0.999}
P_E = {True: 0.002, False: 0.998}

P_A_given_B_E = {
    (True, True): {True: 0.95, False: 0.05},
    (True, False): {True: 0.94, False: 0.06},
    (False, True): {True: 0.29, False: 0.71},
    (False, False): {True: 0.001, False: 0.999}
}

P_J_given_A = {
    True: {True: 0.90, False: 0.10},
    False: {True: 0.05, False: 0.95}
}

P_M_given_A = {
    True: {True: 0.70, False: 0.30},
    False: {True: 0.01, False: 0.99}
}

# Functions to get probabilities
def P_Alarm(B, E):
    return P_A_given_B_E[(B, E)]

def P_JohnCalls(A):
    return P_J_given_A[A]

def P_MaryCalls(A):
    return P_M_given_A[A]

# Exact inference functions for different queries
def P_JohnCalls_given_Burglary_Earthquake(B, E):
    P_A = P_Alarm(B, E)
    P_J_given_A = P_JohnCalls(True)
    P_not_J_given_A = P_JohnCalls(False)
    return (P_A[True] * P_J_given_A[True]) + (P_A[False] * P_not_J_given_A[False])

def P_Alarm_given_Burglary(B):
    total_prob = 0
    for E in [True, False]:
        total_prob += P_E[E] * P_Alarm(B, E)[True]
    return total_prob

def P_Earthquake_given_MaryCalls(M):
    total_prob = 0
    for A in [True, False]:
        if M:
            total_prob += P_MaryCalls(A)[True]
        else:
            total_prob += P_MaryCalls(A)[False]
    return total_prob

def P_Burglary_given_Alarm(A):
    total_prob = 0
    for E in [True, False]:
        P_A_given_B_E = P_Alarm(True, E)
        total_prob += P_E[E] * P_B[True] * P_A_given_B_E[A]
    return total_prob

def check_conditional_independence(A):
    P_J_given_A = P_JohnCalls(A)[True]
    P_M_given_A = P_MaryCalls(A)[True]
    joint_prob = P_J_given_A * P_M_given_A
    marginal_product = P_J_given_A * P_M_given_A
    return joint_prob, marginal_product

result_a = P_JohnCalls_given_Burglary_Earthquake(B=True, E=False)
print("P(John Calls | Burglary, Earthquake):", result_a)

result_b = P_Alarm_given_Burglary(B=True)
print("P(Alarm | Burglary):", result_b)

result_c = P_Earthquake_given_MaryCalls(M=True)
print("P(Earthquake | Mary Calls):", result_c)

result_d = P_Burglary_given_Alarm(A=True)
print("P(Burglary | Alarm):", result_d)

joint_prob, marginal_product = check_conditional_independence(A=True)
print(f"P(John Calls=True, Mary Calls=True | Alarm=True) = {joint_prob}")
print(f"P(John Calls=True | Alarm=True) * P(Mary Calls=True | Alarm=True) = {marginal_product}")

if abs(joint_prob - marginal_product) < 1e-5:
    print("John Calls is conditionally independent of Mary Calls given Alarm.")
else:
    print("John Calls is NOT conditionally independent of Mary Calls given Alarm.")


P(John Calls | Burglary, Earthquake): 0.903
P(Alarm | Burglary): 0.94002
P(Earthquake | Mary Calls): 0.71
P(Burglary | Alarm): 0.00094002
P(John Calls=True, Mary Calls=True | Alarm=True) = 0.63
P(John Calls=True | Alarm=True) * P(Mary Calls=True | Alarm=True) = 0.63
John Calls is conditionally independent of Mary Calls given Alarm.


# Week 12 - Sampling for Bayesian Networks


In [ ]:
import random

random.seed(42)

P_A = {True: 0.3, False: 0.7}
P_B_given_A = {
    True: {True: 0.1, False: 0.9},
    False: {True: 0.6, False: 0.4}
}
P_E = {True: 0.35, False: 0.65}
P_C_given_A_B = {
    (True, True): {True: 0.05, False: 0.95},
    (True, False): {True: 0.5, False: 0.5},
    (False, True): {True: 0.45, False: 0.55},
    (False, False): {True: 0.6, False: 0.4}
}
P_D_given_C_E = {
    (True, True): {True: 0.01, False: 0.99},
    (True, False): {True: 0.5, False: 0.5},
    (False, True): {True: 0.75, False: 0.25},
    (False, False): {True: 0.31, False: 0.69}
}

def sample_a():
    return random.random() < P_A[True]

def sample_b(a):
    return random.random() < P_B_given_A[a][True]

def sample_e():
    return random.random() < P_E[True]

def sample_c(a, b):
    return random.random() < P_C_given_A_B[(a, b)][True]

def sample_d(c, e):
    return random.random() < P_D_given_C_E[(c, e)][True]

def rejection_sampling(num_samples, evidence):
    samples = []
    for i in range(num_samples):
        a = sample_a()
        b = sample_b(a)
        e = sample_e()
        c = sample_c(a, b)
        d = sample_d(c, e)

        sample = {'A': a, 'B': b, 'E': e, 'C': c, 'D': d}
        if all(sample[var] == evidence[var] for var in evidence):
            samples.append(sample)
        else:
            i -= 1
    return samples

def likelihood_weighting(num_samples, evidence):
    weighted_samples = []
    for _ in range(num_samples):
        weight = 1.0
        a = sample_a()
        if 'A' in evidence:
            weight *= P_A[evidence['A']]

        b = sample_b(a)
        if 'B' in evidence:
            weight *= P_B_given_A[a][evidence['B']]

        e = sample_e()
        if 'E' in evidence:
            weight *= P_E[evidence['E']]

        c = sample_c(a, b)
        if 'C' in evidence:
            weight *= P_C_given_A_B[(a, b)][evidence['C']]

        d = sample_d(c, e)
        if 'D' in evidence:
            weight *= P_D_given_C_E[(c, e)][evidence['D']]

        weighted_samples.append(({'A': a, 'B': b, 'E': e, 'C': c, 'D': d}, weight))
    return weighted_samples

def gibbs_sampling(num_samples, evidence):
    state = {var: random.choice([True, False]) for var in ['A', 'B', 'C', 'D', 'E']}
    state.update(evidence)
    samples = []

    for _ in range(num_samples):
        # Update 'A' using its Markov Blanket: {B, C}
        if 'A' not in evidence:
            prob_a_true = P_A[True] * P_B_given_A[True][state['B']] * P_C_given_A_B[(True, state['B'])][state['C']]
            prob_a_false = P_A[False] * P_B_given_A[False][state['B']] * P_C_given_A_B[(False, state['B'])][state['C']]
            state['A'] = random.random() < (prob_a_true / (prob_a_true + prob_a_false))

        # Update 'E' using its Markov Blanket: {D}
        if 'E' not in evidence:
            prob_e_true = P_E[True] * P_D_given_C_E[(state['C'], True)][state['D']]
            prob_e_false = P_E[False] * P_D_given_C_E[(state['C'], False)][state['D']]
            state['E'] = random.random() < (prob_e_true / (prob_e_true + prob_e_false))

        # Update 'B' using its Markov Blanket: {A, C}
        if 'B' not in evidence:
            prob_b_true = P_B_given_A[state['A']][True] * P_C_given_A_B[(state['A'], True)][state['C']]
            prob_b_false = P_B_given_A[state['A']][False] * P_C_given_A_B[(state['A'], False)][state['C']]
            state['B'] = random.random() < (prob_b_true / (prob_b_true + prob_b_false))

        # Update 'C' using its Markov Blanket: {A, B, D}
        if 'C' not in evidence:
            prob_c_true = P_C_given_A_B[(state['A'], state['B'])][True] * P_D_given_C_E[(True, state['E'])][state['D']]
            prob_c_false = P_C_given_A_B[(state['A'], state['B'])][False] * P_D_given_C_E[(False, state['E'])][state['D']]
            state['C'] = random.random() < (prob_c_true / (prob_c_true + prob_c_false))

        # Update 'D' using its Markov Blanket: {C, E}
        if 'D' not in evidence:
            prob_d_true = P_D_given_C_E[(state['C'], state['E'])][True]
            prob_d_false = P_D_given_C_E[(state['C'], state['E'])][False]
            state['D'] = random.random() < (prob_d_true / (prob_d_true + prob_d_false))

        # Collect the current state
        samples.append(state.copy())

    return samples
def calculate_probability(samples, query_var, query_val):
    total_weight = sum(weight for _, weight in samples)
    query_weight = sum(weight for sample, weight in samples if sample[query_var] == query_val)
    return query_weight / total_weight if total_weight > 0 else 0

def calculate_probability_rejection_or_gibbs(samples, query_var, query_val):
    if not samples:
        return 0
    count_query = sum(1 for sample in samples if sample[query_var] == query_val)
    return count_query / len(samples)


num_samples = 10000

evidence_1 = {'C': True}
samples_1 = likelihood_weighting(num_samples, evidence_1)
query_1_result = calculate_probability(samples_1, 'A', True)
print(f"P(High Income | Payment = payback) = {query_1_result:.2f}")

evidence_2 = {'A': False, 'D': False}
samples_2 = likelihood_weighting(num_samples, evidence_2)
query_2_result = calculate_probability(samples_2, 'E', False)
print(f"P(Housing = Tenant | Low Income, No security) = {query_2_result:.2f}")

evidence_3 = {'B': True}
samples_3 = likelihood_weighting(num_samples, evidence_3)
query_3_result = calculate_probability(samples_3, 'C', False)
print(f"P(Payment = default | Deposit = Large) = {query_3_result:.2f}")
print()
evidence_1 = {'C': True}
samples_1 = rejection_sampling(num_samples, evidence_1)
query_1_result = calculate_probability_rejection_or_gibbs(samples_1, 'A', True)
print(f"P(High Income | Payment = payback) = {query_1_result:.2f}")


evidence_2 = {'A': False, 'D': False}
samples_2 = rejection_sampling(num_samples, evidence_2)
query_2_result = calculate_probability_rejection_or_gibbs(samples_2, 'E', False)
print(f"P(Housing = Tenant | Low Income, No security) = {query_2_result:.2f}")

evidence_3 = {'B': True}
samples_3 = rejection_sampling(num_samples, evidence_3)
query_3_result = calculate_probability_rejection_or_gibbs(samples_3, 'C', False)
print(f"P(Payment = default | Deposit = Large) = {query_3_result:.2f}")
print()
samples_1 = gibbs_sampling(num_samples, evidence_1)
query_1_result = calculate_probability_rejection_or_gibbs(samples_1, 'A', True)
print(f"P(High Income | Payment = payback) = {query_1_result:.2f}")

evidence_2 = {'A': False, 'D': False}
samples_2 = gibbs_sampling(num_samples, evidence_2)
query_2_result = calculate_probability_rejection_or_gibbs(samples_2, 'E', False)
print(f"P(Housing = Tenant | Low Income, No security) = {query_2_result:.2f}")


evidence_3 = {'B': True}
samples_3 = gibbs_sampling(num_samples, evidence_3)
query_3_result = calculate_probability_rejection_or_gibbs(samples_3, 'C', False)
print(f"P(Payment = default | Deposit = Large) = {query_3_result:.2f}")


P(High Income | Payment = payback) = 0.27
P(Housing = Tenant | Low Income, No security) = 0.64
P(Payment = default | Deposit = Large) = 0.50

P(High Income | Payment = payback) = 0.27
P(Housing = Tenant | Low Income, No security) = 0.65
P(Payment = default | Deposit = Large) = 0.57

P(High Income | Payment = payback) = 0.28
P(Housing = Tenant | Low Income, No security) = 0.64
P(Payment = default | Deposit = Large) = 0.58


In [ ]:
data = {
    (0, 0, 1, 0, 0): 5,
    (0, 0, 0, 1, 1): 1,
    (0, 0, 1, 1, 1): 1,
    (0, 0, 0, 0, 0): 50,
    (0, 1, 1, 0, 1): 1,
    (0, 1, 0, 1, 0): 1,
    (0, 1, 0, 0, 1): 1,
    (0, 1, 0, 1, 1): 10,
    (1, 0, 0, 1, 0): 1,
    (1, 0, 1, 0, 1): 8,
    (1, 0, 1, 1, 1): 15,
    (1, 1, 0, 0, 1): 1,
    (1, 1, 0, 1, 0): 2,
    (1, 1, 0, 1, 1): 15,
    (1, 1, 1, 1, 1): 29,
    (1, 1, 1, 0, 0): 0,
}

prior_A = 0.5
prior_B = 0.5

def calculate_mle(query, given):
    match_count = 0
    total_count = 0
    for (A, B, C, D, E), count in data.items():
        record = {'A': A, 'B': B, 'C': C, 'D': D, 'E': E}
        if all(record[var] == val for var, val in given.items()):
            total_count += count
            if all(record[var] == val for var, val in query.items()):
                match_count += count
    return match_count / total_count if total_count > 0 else 0

def calculate_map(query, given, prior):
    mle = calculate_mle(query, given)
    return (mle * prior) / ((mle * prior) + ((1 - mle) * (1 - prior)))

mle_1 = calculate_mle({"A": 1}, {"C": 1, "D": 1})
map_1 = calculate_map({"A": 1}, {"C": 1, "D": 1}, prior_A)

mle_2 = calculate_mle({"A": 1, "B": 1}, {"C": 1})
map_2 = calculate_map({"A": 1, "B": 1}, {"C": 1}, prior_A * prior_B)

mle_3 = calculate_mle({"C": 1}, {"D": 1, "E": 1})
map_3 = calculate_map({"C": 1}, {"D": 1, "E": 1}, 0.5)

mle_4 = calculate_mle({"B": 1, "C": 1}, {"D": 1, "E": 1})
map_4 = calculate_map({"B": 1, "C": 1}, {"D": 1, "E": 1}, prior_B * 0.5)

mle_5 = calculate_mle({"D": 1, "E": 1}, {"C": 1})
map_5 = calculate_map({"D": 1, "E": 1}, {"C": 1}, 0.5 * 0.5)

print("P(A | C=1, D=1) - MLE:", mle_1, "MAP:", map_1)
print("P(A=1, B=1 | C=1) - MLE:", mle_2, "MAP:", map_2)
print("P(C=1 | D=1, E=1) - MLE:", mle_3, "MAP:", map_3)
print("P(B=1, C=1 | D=1, E=1) - MLE:", mle_4, "MAP:", map_4)
print("P(D=1, E=1 | C=1) - MLE:", mle_5, "MAP:", map_5)

P(A | C=1, D=1) - MLE: 0.9777777777777777 MAP: 0.9777777777777777
P(A=1, B=1 | C=1) - MLE: 0.4915254237288136 MAP: 0.24369747899159666
P(C=1 | D=1, E=1) - MLE: 0.6338028169014085 MAP: 0.6338028169014085
P(B=1, C=1 | D=1, E=1) - MLE: 0.4084507042253521 MAP: 0.18709677419354842
P(D=1, E=1 | C=1) - MLE: 0.7627118644067796 MAP: 0.5172413793103448


# Week 13 - EM Algorithm

In [ ]:
import numpy as np
import pandas as pd

# Re-define and refine the EM algorithm with corrections
def gaussian_pdf(x, mean, cov):
    """Compute Gaussian PDF for a single data point."""
    size = len(x)
    det_cov = np.linalg.det(cov)
    inv_cov = np.linalg.inv(cov)
    norm_const = 1.0 / (np.sqrt((2 * np.pi) ** size * det_cov))
    diff = x - mean
    result = np.exp(-0.5 * np.dot(diff.T, np.dot(inv_cov, diff)))
    return norm_const * result

def em_algorithm(X, means, covariances, weights, k, max_iters=100, tol=1e-4):
    """EM algorithm with improvements in log-likelihood calculation and convergence check."""
    n, d = X.shape
    log_likelihoods = []

    for iteration in range(max_iters):
        # E-step: Calculate responsibilities
        responsibilities = np.zeros((n, k))
        for i in range(n):
            for j in range(k):
                responsibilities[i, j] = weights[j] * gaussian_pdf(X[i], means[j], covariances[j])
        responsibilities /= responsibilities.sum(axis=1, keepdims=True)  # Normalize

        # M-step: Update parameters
        Nk = responsibilities.sum(axis=0)  # Total responsibility for each cluster
        weights = Nk / n  # Update weights
        means = np.dot(responsibilities.T, X) / Nk[:, None]  # Update means

        covariances = []
        for j in range(k):
            # Update covariance matrix for each cluster
            diff = X - means[j]
            cov = np.dot((responsibilities[:, j] * diff.T), diff) / Nk[j]
            covariances.append(cov + 1e-6 * np.eye(d))  # Add small regularization term

        # Log-likelihood calculation for convergence check
        log_likelihood = 0
        for i in range(n):
            cluster_log_probs = sum(weights[j] * gaussian_pdf(X[i], means[j], covariances[j]) for j in range(k))
            log_likelihood += np.log(cluster_log_probs)
        log_likelihoods.append(log_likelihood)

        # Convergence check based on log-likelihood
        if iteration > 0 and abs(log_likelihood - log_likelihoods[-2]) < tol:
            print(f"Convergence reached at iteration {iteration}.")
            break

    return means, covariances, weights, responsibilities, log_likelihoods

# Load and process data
data_path = 'cluster_data.csv'
data = pd.read_csv(data_path)
X = data.values  # Convert to numpy array for easier matrix operations
n, d = X.shape  # Number of data points and dimensions
k = 3  # Number of clusters

# Initialize parameters
np.random.seed(42)
means = X[np.random.choice(n, k, replace=False)]  # Randomly initialize means
covariances = [np.eye(d) for _ in range(k)]  # Start with identity matrices for covariance
weights = np.ones(k) / k  # Equal weights for each cluster

# Run the EM algorithm
means, covariances, weights, responsibilities, log_likelihoods = em_algorithm(X, means, covariances, weights, k)

# Display the final results of the EM algorithm in a readable format with 4 decimal places
print("Final EM Algorithm Results:\n")

print("Cluster Means:")
for i, mean in enumerate(means, start=1):
    formatted_mean = [f"{value:.4f}" for value in mean]
    print(f"  Cluster {i}: {formatted_mean}")

print("\nCovariance Matrices:")
for i, cov in enumerate(covariances, start=1):
    formatted_cov = [[f"{value:.4f}" for value in row] for row in cov]
    print(f"  Cluster {i}:\n{formatted_cov}\n")

print("Mixing Coefficients (Weights):")
for i, weight in enumerate(weights, start=1):
    print(f"  Cluster {i}: {weight:.4f}")

print(f"\nFinal Log-Likelihood: {log_likelihoods[-1]:.4f}")
print(f"Number of Iterations until Convergence: {len(log_likelihoods)}")



Convergence reached at iteration 32.
Final EM Algorithm Results:

Cluster Means:
  Cluster 1: ['5.9151', '2.7779', '4.2017', '1.2970']
  Cluster 2: ['5.0041', '3.4163', '1.4653', '0.2449']
  Cluster 3: ['6.5447', '2.9487', '5.4798', '1.9847']

Covariance Matrices:
  Cluster 1:
[['0.2753', '0.0969', '0.1847', '0.0544'], ['0.0969', '0.0926', '0.0911', '0.0430'], ['0.1847', '0.0911', '0.2007', '0.0610'], ['0.0544', '0.0430', '0.0610', '0.0320']]

  Cluster 2:
[['0.1241', '0.1001', '0.0163', '0.0106'], ['0.1001', '0.1450', '0.0118', '0.0115'], ['0.0163', '0.0118', '0.0300', '0.0056'], ['0.0106', '0.0115', '0.0056', '0.0115']]

  Cluster 3:
[['0.3870', '0.0922', '0.3028', '0.0616'], ['0.0922', '0.1103', '0.0843', '0.0560'], ['0.3028', '0.0843', '0.3277', '0.0744'], ['0.0616', '0.0560', '0.0744', '0.0857']]

Mixing Coefficients (Weights):
  Cluster 1: 0.3013
  Cluster 2: 0.3289
  Cluster 3: 0.3698

Final Log-Likelihood: -182.5256
Number of Iterations until Convergence: 33


In [ ]:
import numpy as np
import pandas as pd

def gaussian_pdf(x, mean, cov):
    size = len(x)
    det_cov = np.linalg.det(cov)
    inv_cov = np.linalg.inv(cov)
    norm_const = 1.0 / (np.sqrt((2 * np.pi) ** size * det_cov))
    diff = x - mean
    return norm_const * np.exp(-0.5 * np.dot(diff.T, np.dot(inv_cov, diff)))

def em_algorithm(X, means, covariances, weights, k, max_iters=100, tol=1e-4):
    n, d = X.shape
    log_likelihoods = []

    for _ in range(max_iters):
        responsibilities = np.zeros((n, k))
        for i in range(n):
            for j in range(k):
                responsibilities[i, j] = weights[j] * gaussian_pdf(X[i], means[j], covariances[j])
        responsibilities /= responsibilities.sum(axis=1, keepdims=True)

        Nk = responsibilities.sum(axis=0)
        weights = Nk / n
        means = np.dot(responsibilities.T, X) / Nk[:, None]

        covariances = []
        for j in range(k):
            diff = X - means[j]
            cov = np.dot((responsibilities[:, j] * diff.T), diff) / Nk[j]
            covariances.append(cov + 1e-6 * np.eye(d))

        log_likelihood = 0
        for i in range(n):
            cluster_log_probs = sum(weights[j] * gaussian_pdf(X[i], means[j], covariances[j]) for j in range(k))
            log_likelihood += np.log(cluster_log_probs)
        log_likelihoods.append(log_likelihood)

        if len(log_likelihoods) > 1 and abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol:
            break

    return means, covariances, weights, responsibilities, log_likelihoods

data = pd.read_csv('cluster_data.csv')
X = data.values
n, d = X.shape
k = 3

np.random.seed(42)
means = X[np.random.choice(n, k, replace=False)]
covariances = [np.eye(d) for _ in range(k)]
weights = np.ones(k) / k

means, covariances, weights, responsibilities, log_likelihoods = em_algorithm(X, means, covariances, weights, k)

print("Cluster Means:")
for mean in means:
    print(mean)

print("\nCovariance Matrices:")
for cov in covariances:
    print(cov)

print("\nMixing Coefficients (Weights):")
print(weights)

print(f"\nFinal Log-Likelihood: {log_likelihoods[-1]}")
print(f"Iterations: {len(log_likelihoods)}")


Cluster Means:
[5.91505737 2.77785152 4.2017337  1.29703708]
[5.00408163 3.41632653 1.46530612 0.24489796]
[6.54466094 2.9487046  5.47977952 1.98474849]

Covariance Matrices:
[[0.27532288 0.0969218  0.18468932 0.05440487]
 [0.0969218  0.09264209 0.0911354  0.04299653]
 [0.18468932 0.0911354  0.20070106 0.06100855]
 [0.05440487 0.04299653 0.06100855 0.03201251]]
[[0.12406597 0.10013744 0.01626406 0.01063307]
 [0.10013744 0.14504057 0.01179092 0.01151187]
 [0.01626406 0.01179092 0.03002182 0.00563932]
 [0.01063307 0.01151187 0.00563932 0.01145456]]
[[0.3870497  0.09220735 0.30277331 0.06160045]
 [0.09220735 0.11034157 0.08426148 0.05599586]
 [0.30277331 0.08426148 0.32767764 0.07442711]
 [0.06160045 0.05599586 0.07442711 0.0857432 ]]

Mixing Coefficients (Weights):
[0.30130917 0.32885906 0.36983177]

Final Log-Likelihood: -182.52558413248104
Iterations: 33


# Bonus ques

In [ ]:
from copy import deepcopy

def dpll(cnf, assignment={}):

    if not cnf:
        return True, assignment

    if any([len(clause) == 0 for clause in cnf]):
        return False, {}

    for clause in cnf:
        if len(clause) == 1:
            literal = clause[0]
            return dpll(unit_propagate(cnf, literal), {**assignment, abs(literal): literal > 0})

    for literal in pure_literals(cnf):
        return dpll(unit_propagate(cnf, literal), {**assignment, abs(literal): literal > 0})

    literal = choose_literal(cnf)

    satisfiable, result = dpll(unit_propagate(cnf, literal), {**assignment, abs(literal): literal > 0})
    if satisfiable:
        return True, result
    return dpll(unit_propagate(cnf, -literal), {**assignment, abs(literal): literal < 0})

def unit_propagate(cnf, literal):

    simplified_cnf = []
    for clause in cnf:
        if literal in clause:
            continue
        new_clause = [l for l in clause if l != -literal]
        simplified_cnf.append(new_clause)
    return simplified_cnf

def pure_literals(cnf):

    literals = {literal for clause in cnf for literal in clause}
    pure = set()
    for literal in literals:
        if -literal not in literals:
            pure.add(literal)
    return pure

def choose_literal(cnf):

    for clause in cnf:
        for literal in clause:
            return literal
    return None

def cnf_to_string(cnf):
    return ' ∧ '.join(['(' + ' ∨ '.join([str(l) for l in clause]) + ')' for clause in cnf])




In [ ]:
cnf1 = [[1, -3, 4], [-1, 2, 5], [-2, -4, 6], [3, 5, -6], [-3, -5], [4, -1], [5, 2, -6], [6, -2], [1, -5, 4], [-1, -6, 3]]
cnf2 = [[1, 2, -3], [3, -4], [4, 5, 6], [-1, 4], [2, -5, 6], [-2, 5], [1, 6, -4], [-6, 3], [-3, -5, 1], [4, -2, 3]]
cnf3 = [[-1, 2, 3], [-2, -3], [1, 4, -5], [-4, 2, 5], [3, -6], [1, -6, -2], [6, 5, -1], [1, -3, 6], [4, -2, 1], [-6, 2, -5]]
cnf4 = [[1, 2, 3], [-1, 4], [-2, 5], [-3, 6], [-4, 7], [-5, 6], [-6, 2], [1, 5, -7], [7, -1], [3, 2, -4]]

cnfs = [cnf1, cnf2, cnf3, cnf4]

for i, cnf in enumerate(cnfs, 1):
    satisfiable, assignment = dpll(cnf)
    print(f"CNF {i} ({cnf_to_string(cnf)}): {'Satisfiable' if satisfiable else 'Unsatisfiable'}, Assignment: {assignment}")

CNF 1 ((1 ∨ -3 ∨ 4) ∧ (-1 ∨ 2 ∨ 5) ∧ (-2 ∨ -4 ∨ 6) ∧ (3 ∨ 5 ∨ -6) ∧ (-3 ∨ -5) ∧ (4 ∨ -1) ∧ (5 ∨ 2 ∨ -6) ∧ (6 ∨ -2) ∧ (1 ∨ -5 ∨ 4) ∧ (-1 ∨ -6 ∨ 3)): Satisfiable, Assignment: {1: True, 4: True, 2: True, 6: True, 3: True, 5: False}
CNF 2 ((1 ∨ 2 ∨ -3) ∧ (3 ∨ -4) ∧ (4 ∨ 5 ∨ 6) ∧ (-1 ∨ 4) ∧ (2 ∨ -5 ∨ 6) ∧ (-2 ∨ 5) ∧ (1 ∨ 6 ∨ -4) ∧ (-6 ∨ 3) ∧ (-3 ∨ -5 ∨ 1) ∧ (4 ∨ -2 ∨ 3)): Satisfiable, Assignment: {1: True, 4: True, 3: True, 6: True, 5: True}
CNF 3 ((-1 ∨ 2 ∨ 3) ∧ (-2 ∨ -3) ∧ (1 ∨ 4 ∨ -5) ∧ (-4 ∨ 2 ∨ 5) ∧ (3 ∨ -6) ∧ (1 ∨ -6 ∨ -2) ∧ (6 ∨ 5 ∨ -1) ∧ (1 ∨ -3 ∨ 6) ∧ (4 ∨ -2 ∨ 1) ∧ (-6 ∨ 2 ∨ -5)): Satisfiable, Assignment: {1: False, 2: False, 4: True, 5: True, 6: False, 3: False}
CNF 4 ((1 ∨ 2 ∨ 3) ∧ (-1 ∨ 4) ∧ (-2 ∨ 5) ∧ (-3 ∨ 6) ∧ (-4 ∨ 7) ∧ (-5 ∨ 6) ∧ (-6 ∨ 2) ∧ (1 ∨ 5 ∨ -7) ∧ (7 ∨ -1) ∧ (3 ∨ 2 ∨ -4)): Satisfiable, Assignment: {1: True, 4: True, 7: True, 2: True, 5: True, 6: True}


Graph coloring using dpll

In [ ]:
def graph_coloring_dpll(graph, k):

    n = len(graph)
    cnf = []

    for i in range(n):
        cnf.append([i * k + j + 1 for j in range(k)])
        # this is for creating literals for assigning a color to the node....and after creating cnf we use dpll for that

    for i in range(n):
        for j in range(k):
            for m in range(j + 1, k):
                cnf.append([-(i * k + j + 1), -(i * k + m + 1)])
                # this is for to avoid multiple colors assigning to one node...this is achieved by adding a new clause that consists the above type negations so that if one color is assigned then it will be true then for new clause to be true other should be false that is it is not assigned...

    for i in range(n):
        for j in range(i + 1, n):
            if graph[i][j]:
                for color in range(k):
                    cnf.append([-(i * k + color + 1), -(j * k + color + 1)])
                    # this is to satisfy that neighbours should not have the same color and the logic is same as above

    satisfiable, assignment = dpll(cnf)

    if satisfiable:
        color_assignment = {}
        for i in range(n):
            for color in range(k):
                if assignment.get(i * k + color + 1, False):
                    color_assignment[i + 1] = color + 1
                    break
        return True, color_assignment
    else:
        return False, {}

graph = [
    [0, 1, 1, 0],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [0, 1, 1, 0]
]

k = 3

result, color_assignment = graph_coloring_dpll(graph, k)
if result:
    print(f"Graph is colorable with {k} colors: {color_assignment}")
else:
    print(f"Graph is not colorable with {k} colors.")



Graph is colorable with 3 colors: {1: 1, 2: 2, 3: 3, 4: 1}


8queens using dpll


In [ ]:
def eight_queen_dpll():

    n = 8
    cnf = []

    for i in range(n):
        cnf.append([i * n + j + 1 for j in range(n)])
        for j in range(n):
            for k in range(j + 1, n):
                cnf.append([-(i * n + j + 1), -(i * n + k + 1)])
                #this is for row constraint that is two queens cannot be placed in one row

    for j in range(n):
        cnf.append([i * n + j + 1 for i in range(n)])
        for i in range(n):
            for k in range(i + 1, n):
                cnf.append([-(i * n + j + 1), -(k * n + j + 1)])
                #this is for coloumn constraint that is two queens cannot be placed in one column

    for i in range(n):
        for j in range(n):
            for k in range(1, n):
                if i + k < n and j + k < n:
                    cnf.append([-(i * n + j + 1), -((i + k) * n + (j + k) + 1)])
                    # this is to check the main diagonal constraint that is if queen is placed at (i,j) then it should not be placed at (i+k,j+k)
                if i + k < n and j - k >= 0:
                    cnf.append([-(i * n + j + 1), -((i + k) * n + (j - k) + 1)])
                    # similarly it checks the anti diagonal constraint that is if queen is placed at (i,j) then it should not be placed at (i+k,j-k)
    satisfiable, assignment = dpll(cnf)

    if satisfiable:
        queen_positions = []
        for i in range(n):
            for j in range(n):
                if assignment.get(i * n + j + 1, False):
                    queen_positions.append((i + 1, j + 1))
                    break
        return True, queen_positions
    else:
        return False, []

result, positions = eight_queen_dpll()
if result:
    print(f"8-Queen problem is solvable. Queen positions: {positions}")
else:
    print(f"8-Queen problem is not solvable.")


8-Queen problem is solvable. Queen positions: [(1, 1), (2, 5), (3, 8), (4, 6), (5, 3), (6, 7), (7, 2), (8, 4)]


node reordering

In [ ]:
import math
import random

BLANK = " "

def print_board(board):
    for row in board:
        print("|".join(row))
        print("-" * 5)

def check_winner(board):
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] and board[i][0] != BLANK:
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] and board[0][i] != BLANK:
            return board[0][i]

    if board[0][0] == board[1][1] == board[2][2] and board[0][0] != BLANK:
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] and board[0][2] != BLANK:
        return board[0][2]

    return None

def is_full(board):
    return all(cell != BLANK for row in board for cell in row)

def evaluate(board):
    winner = check_winner(board)
    if winner == "X":
        return 1
    elif winner == "O":
        return -1
    return 0

def get_actions(board, player):
    actions = set()
    for i in range(3):
        for j in range(3):
            if board[i][j] == BLANK:
                actions.add((i, j))
    if len(actions) <= 4:
        return actions
    return rearrange_actions(actions, board, player)

def rearrange_actions(actions, board, player):
    res = []
    for r, c in actions:
        possible_wins = 0
        for i in range(3):
            if i != r and board[i][c] == player:
                possible_wins += 1
            if i != c and board[r][i] == player:
                possible_wins += 1

            if r == c and board[i][i] == player:
                possible_wins += 1
            if r == 2 - c and board[i][2 - i] == player:
                possible_wins += 1
        res.append((possible_wins, (r, c)))
    res.sort(key=lambda x: x[0], reverse=True)
    return [x[1] for x in res]

def minimax_alpha_beta(board, depth, is_maximizing, alpha, beta):
    score = evaluate(board)
    if score == 1 or score == -1 or is_full(board):
        return score

    player = "X" if is_maximizing else "O"
    actions = get_actions(board, player)

    if is_maximizing:
        best_score = -math.inf
        for i, j in actions:
            board[i][j] = "X"
            score = minimax_alpha_beta(board, depth + 1, False, alpha, beta)
            board[i][j] = BLANK
            best_score = max(score, best_score)
            alpha = max(alpha, best_score)
            if beta <= alpha:
                break
        return best_score
    else:
        best_score = math.inf
        for i, j in actions:
            board[i][j] = "O"
            score = minimax_alpha_beta(board, depth + 1, True, alpha, beta)
            board[i][j] = BLANK
            best_score = min(score, best_score)
            beta = min(beta, best_score)
            if beta <= alpha:
                break
        return best_score

def find_best_move(board):
    best_score = -math.inf
    best_move = (-1, -1)

    actions = get_actions(board, "X")
    for i, j in actions:
        board[i][j] = "X"
        score = minimax_alpha_beta(board, 0, False, -math.inf, math.inf)
        board[i][j] = BLANK
        if score > best_score:
            best_score = score
            best_move = (i, j)

    return best_move

def play_game():
    board = [[BLANK] * 3 for _ in range(3)]
    current_player = "Computer"

    for turn in range(9):
        print_board(board)

        winner = check_winner(board)
        if winner:
            print(f"{winner} wins!")
            return

        if is_full(board):
            print("It's a draw!")
            return

        if current_player == "Computer":
            i, j = find_best_move(board)
            board[i][j] = "X"
            current_player = "Human"
        else:
            print("Your turn! Enter row and column (0-2):")
            i, j = map(int, input().split())
            if board[i][j] == BLANK:
                board[i][j] = "O"
                current_player = "Computer"
            else:
                print("Invalid move, try again.")
                continue

    print_board(board)
    if not check_winner(board):
        print("It's a draw!")

# Start the game
play_game()
